### Load tensorflow

In [0]:
import tensorflow as tf

In [0]:
tf.__version__

'2.0.0-beta1'

### Enable Eager Execution if you are using tensorflow 1.x

In [0]:
#To Enable Eager Execution
tf.enable_eager_execution()

### Collect Data

In [0]:
import pandas as pd

In [0]:
data =  pd.read_csv('prices.csv')

In [11]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Check all columns in the dataset

In [12]:
data.shape

(229388, 7)

In [14]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [27]:
##data.drop(columns=['date'],axis=1)

data_dp = data.drop(labels = ['date','symbol'], axis = 1)
data_dp.head()

 

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


In [25]:
data_dp.head()

,date,open,close,low,high,volume
0,2016-01-05 00:00:00,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,117.010002,114.970001,114.089996,117.330002,1408600.0


In [21]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [29]:
dp1 = data_dp.head(1000)

dp1.shape

(1000, 5)

### Convert Float64 to Float32

In [126]:

dp2= dp1.astype('float32')

dp2.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
x=dp2.drop('close',axis=1)
y=dp2['close']
train_x, test_x,train_y,test_y = train_test_split(x, y, test_size=.3, random_state=7)

In [128]:
train_x.shape

(700, 4)

In [0]:
TEST_Y = np.array(train_y)

### Normalize Train and Test Data 

In [0]:

import numpy as np
transformer = Normalizer()
x_train= transformer.fit_transform(train_x)
y_train=np.array(train_y)

## Building the graph in tensorflow

2.Define Weights and Bias

In [0]:
#We are initializing weights and Bias with Zero
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

3.Prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

4.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b


## Execute the Graph for 100 epochs and observe the loss

In [136]:
for i in range(100):
    
    w, b = train(x_train,y_train, w, b)
    
    print('Current Loss on iteration', i, loss(y_train, prediction(x_train, w, b)).numpy())

Current Loss on iteration 0 7663.5674
Current Loss on iteration 1 7351.068
Current Loss on iteration 2 7063.063
Current Loss on iteration 3 6797.6484
Current Loss on iteration 4 6553.0317
Current Loss on iteration 5 6327.598
Current Loss on iteration 6 6119.837
Current Loss on iteration 7 5928.3633
Current Loss on iteration 8 5751.9062
Current Loss on iteration 9 5589.2773
Current Loss on iteration 10 5439.401
Current Loss on iteration 11 5301.2764
Current Loss on iteration 12 5173.9775
Current Loss on iteration 13 5056.6597
Current Loss on iteration 14 4948.547
Current Loss on iteration 15 4848.9014
Current Loss on iteration 16 4757.0674
Current Loss on iteration 17 4672.435
Current Loss on iteration 18 4594.4414
Current Loss on iteration 19 4522.562
Current Loss on iteration 20 4456.3125
Current Loss on iteration 21 4395.264
Current Loss on iteration 22 4338.998
Current Loss on iteration 23 4287.1445
Current Loss on iteration 24 4239.3525
Current Loss on iteration 25 4195.312
Current

### Get the shapes and values of W and b

In [137]:
#Check Weights and Bias
print('Weights:\n', w.numpy())
print('Bias:\n',b.numpy())

Weights:
 [[2.5370640e-03]
 [2.5125609e-03]
 [2.5582323e-03]
 [3.3697968e+01]]
Bias:
 [33.697968]


### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [0]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(1, input_shape=(4,)))

#Compile the model
model.compile(optimizer='sgd', loss='mse')
              
              

### Execute the model

In [146]:
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
700/700 [==============================] - 0s 189us/sample - loss: 5960.2531
Epoch 2/10
700/700 [==============================] - 0s 39us/sample - loss: 4063.5770
Epoch 3/10
700/700 [==============================] - 0s 38us/sample - loss: 3742.1475
Epoch 4/10
700/700 [==============================] - 0s 36us/sample - loss: 3691.6651
Epoch 5/10
700/700 [==============================] - 0s 39us/sample - loss: 3683.2355
Epoch 6/10
700/700 [==============================] - 0s 32us/sample - loss: 3682.6257
Epoch 7/10
700/700 [==============================] - 0s 38us/sample - loss: 3680.5913
Epoch 8/10
700/700 [==============================] - 0s 35us/sample - loss: 3682.0956
Epoch 9/10
700/700 [==============================] - 0s 36us/sample - loss: 3680.8242
Epoch 10/10
700/700 [==============================] - 0s 41us/sample - loss: 3681.9083


### Classification using Keras 

In [147]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 1)                 5         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________


In [148]:
model.get_weights()

[array([[ 0.6482708 ],
        [ 0.5859129 ],
        [-0.76487297],
        [34.12465   ]], dtype=float32), array([34.148727], dtype=float32)]

### Load the given Iris data using pandas (Iris.csv)

In [150]:
iris = pd.read_csv('Iris-2.csv')
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [151]:
iris.dtypes

Id                 int64
SepalLengthCm    float64
SepalWidthCm     float64
PetalLengthCm    float64
PetalWidthCm     float64
Species           object
dtype: object

In [0]:
iris.drop("Id",axis=1,inplace=True)

### Splitting the data into feature set and target set

In [0]:
features = iris.drop(columns= ['Species']) 

target = iris[['Species']]

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
from sklearn.preprocessing import LabelEncoder

Y_processed = pd.DataFrame(target).apply(LabelEncoder().fit_transform)

testY = tf.keras.utils.to_categorical(Y_processed, num_classes=3)

In [156]:
testY

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0

### Divide the dataset into Training and test (70:30)

In [0]:
from sklearn.model_selection import train_test_split
transformer = Normalizer()


train_x, test_x,train_y,test_y = train_test_split(features, testY, test_size=.3, random_state=7)

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [0]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dense(10, activation='relu', input_shape=(4,)))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

#Create optimizer with non-default learning rate
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)

#Compile the model
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

### Fitting the model and predicting 

In [159]:
model.fit(train_x.values, train_y, validation_split=0.2, epochs=30)

Train on 84 samples, validate on 21 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/30
84/84 [==============================] - 0s 5ms/sample - loss: 1.2027 - acc: 0.0714 - val_loss: 1.6855 - val_acc: 0.3810
Epoch 2/30
84/84 [==============================] - 0s 164us/sample - loss: 1.1259 - acc: 0.2381 - val_loss: 1.5128 - val_acc: 0.3810
Epoch 3/30
84/84 [==============================] - 0s 150us/sample - loss: 1.0663 - acc: 0.3571 - val_loss: 1.3764 - val_acc: 0.3810
Epoch 4/30
84/84 [==============================] - 0s 139us/sample - loss: 1.0088 - acc: 0.3571 - val_loss: 1.2921 - val_acc: 0.3810
Epoch 5/30
84/84 [==============================] - 0s 171us/sample - loss: 0.9801 - acc: 0.4167 - val_loss: 1.2233 - val_acc: 0.3810
Epoch 6/30
84/84 [==============================] - 0s 155us/sample - loss: 0.9423 - acc: 0.5357 - val_loss: 1.1679 - val_acc: 0.3810
Epoch 7/30
84/84 [==============================] - 0s 156us

### Report Accuracy of the predicted values

In [160]:
model.evaluate(test_x,test_y)

45/45 [==============================] - 0s 2ms/sample - loss: 1.1745 - acc: 0.4000


[1.1745498710208468, 0.4]

In [161]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo multiple                  16        
_________________________________________________________________
dense_4 (Dense)              multiple                  50        
_________________________________________________________________
dense_5 (Dense)              multiple                  88        
_________________________________________________________________
dense_6 (Dense)              multiple                  27        
Total params: 181
Trainable params: 173
Non-trainable params: 8
_________________________________________________________________
